# Correlate daily climatology of cross slope heat transport with ASC

This is the original code created by Adele Morrison. I Calculates the Annual mean heat transport Crossing the 1000m isobath. An eddited version of this code was used (check ../python/Daily_cross_slope_heat_transport_paralel.py) to calculate the transports monthly, and run as a python script on Gadi.

In [1]:
# # Compute monthly ASC along contour and save

import cosima_cookbook as cc
from cosima_cookbook import distributed as ccd
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr
import glob,os
import cmocean.cm as cmocean
import os
import sys
import glob

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)
logging.getLogger('distributed.utils_perf').setLevel(logging.ERROR)

from dask.distributed import Client

import warnings # ignore these warnings
warnings.filterwarnings("ignore", category = FutureWarning)
warnings.filterwarnings("ignore", category = UserWarning)
warnings.filterwarnings("ignore", category = RuntimeWarning)



In [2]:
client = Client(n_workers=224)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 224
Total threads: 224,Total memory: 250.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37673,Workers: 224
Dashboard: /proxy/8787/status,Total threads: 224
Started: Just now,Total memory: 250.00 GiB
Comm: tcp://127.0.0.1:44499,Total threads: 1
Dashboard: /proxy/34467/status,Memory: 1.12 GiB
Nanny: tcp://127.0.0.1:37255,


In [3]:
	session = cc.database.create_session()
	nnmonth=8    
	monthdays = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
	month = str(int(nnmonth))
	month = month.zfill(2)
	year = str(2170)
	start_time=year+'-'+month  
	 #Start_time0 and end_time0 are for importing the daily transport, and it ahsthe number of days in the month    
	start_time0=year+'-'+month +'-01'     
	end_time0=year+'-'+month +'-' + str(monthdays[np.int(np.int(nnmonth)-1)])
	print(start_time0) 
	print(end_time0) 
	exp = '01deg_jra55v13_ryf9091'
	
	print("Start date =" + start_time) 
	year2=str(np.int(start_time[0:4])+1)
	month2=str(np.int(start_time[5:7])+1)
	month2=str(int(month2))
	month2 = month2.zfill(2)    
	print("month2 is =" + month2) 
	print("year2 is =" + str(year2))     
	
	imon = np.int(1)
	if imon <12:
	    end_time=year+'-'+month2
	else:
	    end_time=year2+'-01'
	    
	print("End date =" + end_time) 
	
	      
	time_period = str(int(start_time[:4]))+'-'+str(int(end_time[:4]))
	
	# reference density value:
	rho_0 = 1035.0
	# specific heat capacity of sea water:
	cp = 3992.1
	lat_range = slice(-90,-59)
	
	isobath_depth = 1000
	
	# pick a freezing point temp:
	temp_freezing = -3.82
	
	
	print("importing isobath mask")
	outfile = '/g/data/v45/akm157/model_data/access-om2/Antarctic_slope_contour_'+str(isobath_depth)+'m.npz'
	data = np.load(outfile)
	mask_y_transport = data['mask_y_transport']
	mask_x_transport = data['mask_x_transport']
	mask_y_transport_numbered = data['mask_y_transport_numbered']
	mask_x_transport_numbered = data['mask_x_transport_numbered']
	
	yt_ocean = cc.querying.getvar(exp,'yt_ocean',session,n=1)
	yt_ocean = yt_ocean.sel(yt_ocean=lat_range)
	yu_ocean = cc.querying.getvar(exp,'yu_ocean',session,n=1)
	yu_ocean = yu_ocean.sel(yu_ocean=lat_range)
	xt_ocean = cc.querying.getvar(exp,'xt_ocean',session,n=1)
	xu_ocean = cc.querying.getvar(exp,'xu_ocean',session,n=1)   
	
	mask_x_transport =xr.DataArray(data['mask_x_transport']).assign_coords({"dim_0": np.array(yt_ocean),"dim_1": np.array(xu_ocean)}).rename(dim_0="yt_ocean",dim_1="xu_ocean")
	mask_y_transport =xr.DataArray(data['mask_y_transport']).assign_coords({"dim_0": np.array(yt_ocean),"dim_1": np.array(xu_ocean)}).rename(dim_0="yu_ocean",dim_1="xt_ocean")
	mask_x_transport_numbered =xr.DataArray(data['mask_x_transport_numbered']).assign_coords({"dim_0": np.array(yt_ocean),"dim_1": np.array(xt_ocean)}).rename(dim_0="yt_ocean",dim_1="xt_ocean")
	mask_y_transport_numbered =xr.DataArray(data['mask_y_transport_numbered']).assign_coords({"dim_0": np.array(yt_ocean),"dim_1": np.array(xt_ocean)}).rename(dim_0="yt_ocean",dim_1="xt_ocean")
	
	num_points = int(np.maximum(np.max(mask_y_transport_numbered),np.max(mask_x_transport_numbered)))
	    
	lat_along_contour = np.zeros((num_points))
	lon_along_contour = np.zeros((num_points))
	
	# locations for zonal transport:
	x_indices_masked = mask_x_transport_numbered.stack().values
	x_indices = np.sort(x_indices_masked[x_indices_masked>0])
	for count in x_indices:
	    count = int(count)
	    jj = int(np.where(mask_x_transport_numbered==count)[0])
	    ii = int(np.where(mask_x_transport_numbered==count)[1])   
	    lon_along_contour[count-1] = xu_ocean[ii].values
	    lat_along_contour[count-1] = mask_x_transport_numbered.yt_ocean[jj].values
	    
	# locations for meridional transport:
	y_indices_masked = mask_y_transport_numbered.stack().values
	y_indices = np.sort(y_indices_masked[y_indices_masked>0])
	for count in y_indices:
	    count = int(count)
	    jj = np.where(mask_y_transport_numbered==count)[0]
	    ii = np.where(mask_y_transport_numbered==count)[1]
	    lon_along_contour[count-1] = mask_x_transport_numbered.xt_ocean[ii].values
	    lat_along_contour[count-1] = yu_ocean[jj].values
	
	# Note temp_yflux_adv is also positioned on north centre edge of t-cell.
	
	print("importing temp_xflux and temp_yflux")
	    
	    
	#importing all heat transports calculated offline
	src_HT='/g/data/x77/wf4500/ASC_project/model_data/access-om2/01deg_jra55v13_ryf9091/daily_heat_transport'
	
	glx =glob.glob(src_HT + "/temp_xflux_from_daily_*******.nc")
	glx=sorted(glx)
	temp_xflux=xr.open_mfdataset(glx, concat_dim='time', combine='nested')
	
	
	gly =glob.glob(src_HT + "/temp_yflux_from_daily_*******.nc")
	gly=sorted(gly)
	temp_yflux=xr.open_mfdataset(gly, concat_dim='time', combine='nested')    
	    
	    
	
	
	
	# Getting only the times that we want
	temp_xflux = temp_xflux.sel(time=slice(start_time0,end_time0)).rename(x_ocean='xu_ocean',y_ocean='yt_ocean')
	temp_yflux = temp_yflux.sel(time=slice(start_time0,end_time0)).rename(x_ocean='xt_ocean',y_ocean='yu_ocean')
	
	
	temp_yflux = temp_yflux.temp_yflux.load()
	temp_xflux = temp_xflux.temp_xflux.load()
	
	print("Saving uhrho_vhrho")
	# save a long term average of vhrho_nt and uhrho_et:
	outpath = '/g/data/x77/wf4500/ASC_project/uhrho_vhrho_'+start_time+'.nc'
	# check if already exists:
	if os.path.exists(outpath):
	    average_transports = xr.open_dataset(outpath)
	    # extract arrays from dataset:
	    uhrho_et = average_transports.uhrho_et
	    vhrho_nt = average_transports.vhrho_nt
	else:
	    vhrho_nt = cc.querying.getvar(exp,'vhrho_nt',session,start_time=start_time, end_time=end_time)
	    uhrho_et = cc.querying.getvar(exp,'uhrho_et',session,start_time=start_time, end_time=end_time)
	#    vhrho_nt = cc.querying.getvar(exp,'vhrho_nt',session,frequency='1 daily')
	#    uhrho_et = cc.querying.getvar(exp,'uhrho_et',session,frequency='1 daily')
	
	    vhrho_nt = vhrho_nt.sel(yt_ocean=lat_range).sel(time=slice(start_time0,end_time0))
	    uhrho_et = uhrho_et.sel(yt_ocean=lat_range).sel(time=slice(start_time0,end_time0))
	
	    vhrho_nttime=vhrho_nt.time
	    
	    
	    outpath = '/g/data/x77/wf4500/ASC_project/model_data/access-om2/'+exp+'/Antarctic_cross_slope/Daily/uhrho_vhrho_'+start_time+'.nc'
	    ds = xr.Dataset({'vhrho_nt': vhrho_nt,'uhrho_et':uhrho_et})
	    del ds.vhrho_nt.attrs['time_bounds']
	    del ds.uhrho_et.attrs['time_bounds']
	    #ds.to_netcdf(outpath)
	    #ds.close()
	print('Daily data being calculated for month =  ...')      
	print(vhrho_nttime)
	import os
	#outpath = '/g/data/x77/wf4500/ASC_project/model_data/access-om2/'+exp+'/Antarctic_cross_slope/Daily/uhrho_vhrho_'+start_time+'.nc'
	#ds = xr.open_dataset(outpath)
	vhrho_nt = ds['vhrho_nt']
	uhrho_et = ds['uhrho_et']
	
	# subtract freezing point heat transport:
	yt_ocean = cc.querying.getvar('01deg_jra55v13_ryf9091','yt_ocean',session,n=1)
	dxu = cc.querying.getvar('01deg_jra55v13_ryf9091','dxu',session,n=1)
	dyt = cc.querying.getvar(exp,'dyt',session,n=1)
	# give dxu and dyt correct coordinates:
	dxu.coords['nj'] = yt_ocean.values
	dxu.coords['ni'] = xt_ocean['xt_ocean'].values
	dxu = dxu.rename(({'ni':'xt_ocean', 'nj':'yt_ocean'}))
	dyt.coords['nj'] = yt_ocean.values
	dyt.coords['ni'] = xt_ocean['xt_ocean'].values
	dyt = dyt.rename(({'ni':'xt_ocean', 'nj':'yt_ocean'}))
	# select latitude range:
	dxu = dxu.sel(yt_ocean=lat_range)
	dyt = dyt.sel(yt_ocean=lat_range)
	
	# Note that in newer mom5 versions this could also be done with ty_trans_int_z, 
	# but there is a problem with this diagnostic in older runs, and even
	# using ty_trans, there is a slight difference. Not sure why?
	
	# Note vhrho_nt is v*dz*1035 and is positioned on north centre edge of t-cell.
	# sum in depth:
	vhrho_nt = vhrho_nt.sum('st_ocean')
	uhrho_et = uhrho_et.sum('st_ocean')
	# convert to transport:
	vhrho_nt = vhrho_nt*dxu/rho_0
	uhrho_et = uhrho_et*dyt/rho_0
	
	# overwrite coords, so we can add the freezing point (with uhrho_et and vhrho_nt) without problems:
	yu_ocean = cc.querying.getvar(exp,'yu_ocean',session,n=1)
	yu_ocean = yu_ocean.sel(yu_ocean=lat_range)
	vhrho_nt.coords['yt_ocean'] = yu_ocean.values
	vhrho_nt = vhrho_nt.rename(({'yt_ocean':'yu_ocean'}))
	uhrho_et.coords['xt_ocean'] = xu_ocean.values
	uhrho_et = uhrho_et.rename(({'xt_ocean':'xu_ocean'}))
	
	freezing_point_heat_trans_zonal = cp*rho_0*uhrho_et*temp_freezing
	freezing_point_heat_trans_meridional = cp*rho_0*vhrho_nt*temp_freezing
	
	print("Removing freezing point heat transport")
	# compare both ways:
	temp_yflux_month = temp_yflux - freezing_point_heat_trans_meridional
	temp_xflux_month = temp_xflux - freezing_point_heat_trans_zonal
	    
	temp_yflux_with_mask_month = np.multiply(np.array(temp_yflux),np.array(mask_y_transport))
	temp_xflux_with_mask_month = np.multiply(np.array(temp_xflux),np.array(mask_x_transport))
	
	
	


2170-08-01
2170-08-31
Start date =2170-08
month2 is =09
year2 is =2171
End date =2170-09
importing isobath mask
importing temp_xflux and temp_yflux
Saving uhrho_vhrho
Daily data being calculated for month =  ...
<xarray.DataArray 'time' (time: 31)>
array([cftime.DatetimeNoLeap(2170, 8, 1, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 8, 2, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 8, 3, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 8, 4, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 8, 5, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 8, 6, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 8, 7, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 8, 8, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 8, 9, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2170, 8, 10, 12, 0, 0, 0, has_year_zero=True),


In [4]:
	from joblib import Parallel, delayed
	def HT_paralel(monthdays,temp_yflux_with_mask_month,temp_xflux_with_mask_month,num_points,x_indices,y_indices,mask_y_transport_numbered,mask_x_transport_numbered,lon_along_contour,temp_xflux_month,temp_yflux_month,n):  
	 
	
	
		#aheat_trans_across_contour=np.zeros([np.int(monthdays[np.int(np.int(1)-1)]),6002])
		#azonal_convergence=np.zeros([np.int(monthdays[np.int(np.int(1)-1)]),1428])
		#across_slope_heat_trans=np.zeros([np.int(monthdays[np.int(np.int(1)-1)]),1428])
		aheat_trans_across_contour={}
		azonal_convergence={}
		across_slope_heat_trans={}
		#for n in np.arange(0,2):#np.arange(0,np.int(monthdays[np.int(np.int(1)-1)])):
		temp_yflux_with_mask = temp_yflux_with_mask_month[n]
		temp_xflux_with_mask =temp_xflux_with_mask_month[n]
		print('Day = '+str(n+1))
	
	
		# multiply by mask to get correct direction into or out of isobath contour:
		heat_trans_across_contour = np.zeros((num_points))
		
		# locations for zonal transport, already calculated indices above:
		print('Extract heat transport along contour')        
		for count in x_indices:
			count = int(count)
			jj = int(np.where(mask_x_transport_numbered==count)[0])
			ii = int(np.where(mask_x_transport_numbered==count)[1])
			heat_trans_across_contour[count-1] += temp_xflux_with_mask[jj,ii]
		    
		# locations for meridional transport, already calculated indices above:
		for count in y_indices:
			count = int(count)
			jj = int(np.where(mask_y_transport_numbered==count)[0])
			ii = int(np.where(mask_y_transport_numbered==count)[1])
			heat_trans_across_contour[count-1] += temp_yflux_with_mask[jj,ii]
		   
		print('Creating bins')        
		# convert to longitude coordinate and average into 3 degree longitude bins:
		# in degrees:
		bin_width = 3
		bin_spacing = 0.25
		lon_west = -280
		lon_east = 80
		
		# new coordinate and midpoints of longitude bins:
		full_lon_coord = np.arange(lon_west,lon_east+bin_spacing,bin_spacing)
		lon_bin_midpoints = np.arange(lon_west+bin_width/2,lon_east-bin_width/2,bin_spacing)
		n_bin_edges = len(full_lon_coord)
		
		# sum into longitude bins:
		# need to be very careful of loops, we can't just mask over longitude values, but instead pick indices 
		# on the isobath contour and sum continously along contour between defined indices.
		# (i.e. lon_along_contour is not monotonic)
		# find points on contour to define edges of longitude bins:
		bin_edge_indices = np.zeros(n_bin_edges)
		for lon_bin in range(n_bin_edges-1):
			# find first isobath point that has the right longitude:
			first_point = np.where(lon_along_contour>=full_lon_coord[lon_bin])[0][0]
			# then find all other isobath points with the same longitude as that first point:
			same_lon_points = np.where(lon_along_contour==lon_along_contour[first_point])[0]
			# we want the most southerly of these points on the same longitude line:
			bin_edge_indices[lon_bin] = same_lon_points[np.argmin(lat_along_contour[same_lon_points])]
		    
		# define east/west edges:
		bin_edge_indices = bin_edge_indices.astype(int)
		bin_edge_indices_west = bin_edge_indices[:-int(bin_width/bin_spacing)-1]
		bin_edge_indices_east = bin_edge_indices[int(bin_width/bin_spacing):-1]
		n_bins = len(bin_edge_indices_west)
		
		# sum heat transport from isobath coord into new longitude coord:
		cross_slope_heat_trans = np.zeros(n_bins)
		for lon_bin in range(n_bins):
			heat_trans_this_bin = heat_trans_across_contour[bin_edge_indices_west[lon_bin]:bin_edge_indices_east[lon_bin]]
			cross_slope_heat_trans[lon_bin] = np.sum(heat_trans_this_bin)
		    
		# find average latitude of each bin, so we can plot back on the isobath:
		lat_bin_midpoints = np.zeros(n_bins)
		for lon_bin in range(n_bins):
			# find nearest isobath point:
			lon_index = np.where(lon_along_contour>=lon_bin_midpoints[lon_bin])[0][0]
			lat_bin_midpoints[lon_bin] = lat_along_contour[lon_index]
		
		
		temp_xflux=temp_xflux_month[n]
		temp_yflux=temp_yflux_month[n]   ## fix in the code
		
		print('Calculating Zonal heat convergence')
		# Need to make sure the zonal boundaries here match exactly with the zonal boundaries used for the 
		# longitude averaging above, by using same bin_edge_indices.
		# Just check if isobath point is on x or y grid. if x, sum up to this point from south.
		# if on y grid, sum zonal transport on nearest u grid point to west.
		
		print('Getting westmost heat transport in the bin')
		zonal_heat_trans_west = np.zeros(n_bins)
		for lon_bin in range(n_bins):
			# west limit:
			# reset these to False:
			on_x_grid = False
			on_y_grid = False
			# mask_x_transport_numbered etc indexing starts from 1 not 0, so add 1:
			isobath_west_index = int(bin_edge_indices_west[lon_bin]+1)
			# check if the point is on the x or y transport grid:
			if len(np.where(mask_x_transport_numbered==isobath_west_index)[0])>0:
				on_x_grid = True
				jj = int(np.where(mask_x_transport_numbered==isobath_west_index)[0])
				ii = int(np.where(mask_x_transport_numbered==isobath_west_index)[1])
			elif len(np.where(mask_y_transport_numbered==isobath_west_index)[0])>0:
				on_y_grid = True
				jj = int(np.where(mask_y_transport_numbered==isobath_west_index)[0])
				ii = int(np.where(mask_y_transport_numbered==isobath_west_index)[1])
			if on_x_grid == True:
				zonal_heat_trans_west[lon_bin] = np.sum(temp_xflux[:jj,ii])
			# in this case we want transport half a grid point to the west:
			elif on_y_grid == True:
				# careful if ii=0, then we need heat trans from lon=80, because at limit of zonal grid
				if ii==0:
					zonal_heat_trans_west[lon_bin] = np.sum(temp_xflux[:jj+1,-1])
				else:
					zonal_heat_trans_west[lon_bin] = np.sum(temp_xflux[:jj+1,ii-1])
		
		zonal_heat_trans_east = np.zeros(n_bins)
		for lon_bin in range(n_bins):
			# east limit:
			# reset these to False:
			on_x_grid = False
			on_y_grid = False
			# mask_x_transport_numbered etc indexing starts from 1 not 0, so add 1:
			isobath_east_index = int(bin_edge_indices_east[lon_bin]+1)
			# check if the point is on the x or y transport grid:
			if len(np.where(mask_x_transport_numbered==isobath_east_index)[0])>0:
				on_x_grid = True
				jj = int(np.where(mask_x_transport_numbered==isobath_east_index)[0])
				ii = int(np.where(mask_x_transport_numbered==isobath_east_index)[1])
			elif len(np.where(mask_y_transport_numbered==isobath_east_index)[0])>0:
				on_y_grid = True
				jj = int(np.where(mask_y_transport_numbered==isobath_east_index)[0])
				ii = int(np.where(mask_y_transport_numbered==isobath_east_index)[1])
			if on_x_grid == True:
				zonal_heat_trans_east[lon_bin] = np.sum(temp_xflux[:jj,ii])
				# in this case we want transport half a grid point to the west:
			elif on_y_grid == True:
				# wrap around to east side of grid:
				if ii==0:
					zonal_heat_trans_east[lon_bin] = np.sum(temp_xflux[:jj+1,-1])
				else:
					zonal_heat_trans_east[lon_bin] = np.sum(temp_xflux[:jj+1,ii-1])
		
		zonal_convergence = zonal_heat_trans_east - zonal_heat_trans_west
		
		
		aheat_trans_across_contour[n]=heat_trans_across_contour
		azonal_convergence[n]=zonal_convergence
		across_slope_heat_trans[n]=cross_slope_heat_trans
		
		#Creating the concatenated matrices along time
		return  [aheat_trans_across_contour,azonal_convergence,across_slope_heat_trans]       




In [5]:
%%time
nndays=monthdays[np.int(np.int(nnmonth)-1)]
results = Parallel(n_jobs=-1)(delayed(HT_paralel)(monthdays,temp_yflux_with_mask_month,temp_xflux_with_mask_month,num_points,x_indices,y_indices,mask_y_transport_numbered,mask_x_transport_numbered,lon_along_contour,temp_xflux_month,temp_yflux_month,n) for n in range(nndays))





Day = 15
Extract heat transport along contour
Creating bins
Calculating Zonal heat convergence
Getting westmost heat transport in the bin
Day = 16
Extract heat transport along contour
Creating bins
Calculating Zonal heat convergence
Getting westmost heat transport in the bin
Day = 19
Extract heat transport along contour
Creating bins
Calculating Zonal heat convergence
Getting westmost heat transport in the bin
Day = 23
Extract heat transport along contour
Creating bins
Calculating Zonal heat convergence
Getting westmost heat transport in the bin
Day = 24
Extract heat transport along contour
Creating bins
Calculating Zonal heat convergence
Getting westmost heat transport in the bin
Day = 25
Extract heat transport along contour
Creating bins
Calculating Zonal heat convergence
Getting westmost heat transport in the bin
Day = 10
Extract heat transport along contour
Creating bins
Calculating Zonal heat convergence
Getting westmost heat transport in the bin
Day = 14
Extract heat transport al

In [6]:
bheat_trans_across_contour=np.zeros([nndays,6002])
bzonal_convergence=np.zeros([nndays,1428])
bcross_slope_heat_trans=np.zeros([nndays,1428])
for idays in range(nndays):
    bheat_trans_across_contour[idays,:]=results[idays][0][idays]
    bzonal_convergence[idays,:]=results[idays][1][idays]
    bcross_slope_heat_trans[idays,:]=results[idays][2][idays]

In [7]:
#pre-saved coords
coords = xr.open_dataset('/g/data/x77/wf4500/ASC_project/cross_slope_transport/Monthly/01deg_jra55v13_ryf9091/Ant_cross_slope_heat_terms_online_1000m_2095-01.nc')

blon_bin_midpoints=np.array(coords.lon_bin_midpoints)
blon_along_contour=np.array(coords.lon_along_contour)
blat_bin_midpoints=np.array(coords.lat_bin_midpoints)
bbin_width=np.array(coords.bin_width)
bbin_spacing=np.array(coords.bin_spacing)

In [8]:
# convert to data arrays, so we can save as netcdf:
zonal_convergence = xr.DataArray(bzonal_convergence)
zonal_convergence = zonal_convergence.rename(dim_0='time',dim_1='lon_bin_midpoints')
zonal_convergence['lon_bin_midpoints']=blon_bin_midpoints
zonal_convergence['time']=vhrho_nttime


cross_slope_heat_trans = xr.DataArray(bcross_slope_heat_trans)
cross_slope_heat_trans = cross_slope_heat_trans.rename(dim_0='time',dim_1='lon_bin_midpoints')
cross_slope_heat_trans['lon_bin_midpoints']=blon_bin_midpoints
cross_slope_heat_trans['time']=vhrho_nttime

heat_trans_across_contour = xr.DataArray(bheat_trans_across_contour)
heat_trans_across_contour = heat_trans_across_contour.rename(dim_0='time',dim_1='lon_along_contour')
heat_trans_across_contour['lon_along_contour']=blon_along_contour
heat_trans_across_contour['time']=vhrho_nttime

ds = xr.Dataset({'zonal_convergence': zonal_convergence,'cross_slope_heat_trans':cross_slope_heat_trans,
                'lon_bin_midpoints':blon_bin_midpoints,'lat_bin_midpoints':blat_bin_midpoints,
                 'bin_width':bbin_width,'bin_spacing':bbin_spacing,'heat_trans_across_contour':heat_trans_across_contour,
                'lon_along_contour':blon_along_contour})
ds

<xarray.Dataset>
Dimensions:                    (lon_bin_midpoints: 1428, time: 31,
                                lat_bin_midpoints: 1428, lon_along_contour: 6002)
Coordinates:
  * lon_bin_midpoints          (lon_bin_midpoints) float64 -278.5 ... 78.25
  * time                       (time) object 2170-08-01 12:00:00 ... 2170-08-...
  * lat_bin_midpoints          (lat_bin_midpoints) float64 -65.63 ... -66.49
  * lon_along_contour          (lon_along_contour) float64 -279.9 ... 80.0
Data variables:
    zonal_convergence          (time, lon_bin_midpoints) float64 1.571e+12 .....
    cross_slope_heat_trans     (time, lon_bin_midpoints) float64 -2.816e+07 ....
    bin_width                  int64 3
    bin_spacing                float64 0.25
    heat_trans_across_contour  (time, lon_along_contour) float64 -2.425e+07 ....

In [9]:
outpath = '/g/data/x77/wf4500/ASC_project/cross_slope_transport/Daily/' + exp + '/Ant_cross_slope_heat_terms_online_'+str(isobath_depth)+'m_'+ start_time+'.nc'
ds.to_netcdf(outpath)
print('file saved in ' + outpath)

file saved in /g/data/x77/wf4500/ASC_project/cross_slope_transport/Daily/01deg_jra55v13_ryf9091/Ant_cross_slope_heat_terms_online_1000m_2170-08.nc
